In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml


In [121]:
url = "https://www.airlinequality.com/airline-reviews/british-airways"


In [15]:
data = requests.get(url).content


with open('b_air.html', 'wb') as f_in:
    f_in.write(data)

with open("b_air.html", "rb") as f_out:
    page = f_out.read()

In [19]:
soup = BeautifulSoup(page, 'html.parser')

In [21]:
soup.find('h2', class_='text_header').text

'"Cabin crew were all fantastic"'

In [57]:
soup.find('div', class_='text_content').text


'✅ Trip Verified |  Easy check in a T5. Galleries south and North lounges packed, only just able to get a seat. Didn’t stay long as B gates lounge much quieter. Flight boarded smoothly and on time. A380 upper deck. Old club world seating which is beginning to age a little. However, like the large toilets on this aircraft type. Cabin crew were all fantastic. One family had tech issues with their seats and entertainment systems which crew worked hard on though sadly without success. Food and drink options were all very good. Entertainment was good. Overall an enjoyable flight.'

In [141]:
soup.find('div', class_="rating-10").span.text

'\n\t\t\t\t\t\t\t\t\t\t\t\t\t5'

In [110]:
soup.find("time").text

'13th August 2023'

In [120]:
soup.find('h3').span.next_sibling.strip(" ()")

'United Kingdom'

In [139]:
pages = 36
page_size = 100
reviews = []
stars = []
date = []
country = []


for i in range(1, pages + 1):
    url = "https://www.airlinequality.com/airline-reviews/british-airways"

    page = requests.get(f"{url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}").content
    soup = BeautifulSoup(page, 'html.parser')
    
    # for item in soup.find_all('h2', class_='text_header'):
    #     reviews.append(item.text)

    for items in soup.find_all('div', class_='text_content'):
        reviews.append(items.text)

    for items in soup.find_all('div', class_="rating-10"):
        try:
            stars.append(items.span.text)
        except:
            print(f"Error on page {i}")
            stars.append('None')
    for items in soup.find_all("time"):
        date.append(items.text)


    for items in soup.find_all('h3'):
        country.append(items.span.next_sibling.strip(" ()"))
            

Error on page 31
Error on page 32
Error on page 32
Error on page 35
Error on page 35


In [144]:
stars = [i for i in stars if '\t' not in i]

In [145]:
print(len(reviews)), print(len(date)), print(len(country)), print(len(stars))

3600
3600
3600
3600


(None, None, None, None)

In [146]:
df = pd.DataFrame({"reviews": reviews, "country": country, "date": date, "stars":stars})

In [147]:
df.head()

,reviews,country,date,stars
0,✅ Trip Verified | Easy check in a T5. Galleri...,United Kingdom,13th August 2023,8
1,"Not Verified | Flight delayed by an hour, it ...",United States,12th August 2023,2
2,Not Verified | The staff are very rude and not...,United Kingdom,11th August 2023,2
3,✅ Trip Verified | Good domestic flight operat...,United Kingdom,8th August 2023,10
4,Not Verified | Failed at all basic travel fund...,Canada,8th August 2023,1


In [149]:
 df.to_csv('ba_reviews.csv')